In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
%matplotlib inline

In [2]:
master = pd.read_csv('C:/Users/19739/Desktop/Money Ball/Money_Ball/Resources/People.csv', parse_dates=['debut', 'finalGame'])
# Currently not caring about pitchers



In [3]:
bbmaster = master[['playerID', 'nameFirst', 'nameLast', 'nameGiven']]
print(len(bbmaster))
bbmaster.head(2)

20370


,playerID,nameFirst,nameLast,nameGiven
0,aardsda01,David,Aardsma,David Allan
1,aaronha01,Hank,Aaron,Henry Louis


In [ ]:
batting = pd.read_csv('data/Batting.csv')
batting.columns

In [ ]:
print(len(batting))
batting.head(2)

In [ ]:
master_batting = pd.merge(master, batting)

In [ ]:
batting.columns

In [ ]:
bats = batting[['playerID', 'yearID','AB', 'H', 'BB', 'HBP', 'SF']]
bats.head()
print(len(bats))
bbdata2 = pd.merge(bbmaster, bats)
bbdata2.head(2)

In [ ]:
salaries = pd.read_csv('data/Salaries.csv')
salaries.columns

In [ ]:
salaries = salaries[['yearID', 'playerID', 'salary']]
print(len(salaries))
salaries.head(2)

In [ ]:
bbdata_sal = pd.merge(bbdata2, salaries)
print(len(bbdata_sal))
bbdata_sal.head()

In [ ]:
# Drop all AB values of 0 or null (average at-bats in MLB is 30/year)
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[(bbdata_sal['AB'] < 15) | \
                                              (bbdata_sal['H']  < 1) | \
                                              (pd.isnull(bbdata_sal['AB']))])

In [ ]:
bbdata_sal['OBP'] = bbdata_sal.apply(lambda row: (row['H'] + row['BB'] + row['HBP']) * 100 / \
                                     (row['AB'] + row['BB'] + row['HBP'] + row['SF']), axis=1)

In [ ]:
bbdata_sal.sort(columns=['OBP'], ascending=False).head(2)

In [ ]:
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[bbdata_sal['salary'] <= 0])
pass

In [ ]:
bbdata_sal['OBP_sal_10k'] = bbdata_sal.apply(lambda row: row['OBP'] * 10000 / row['salary'], axis=1)
bbdata_sal.sort(columns='OBP_sal_10k', ascending=False).head(2)

In [ ]:
appearances = pd.read_csv('data/Appearances.csv')
appearances.columns

In [ ]:
appearances.pop('lgID')
appearances.pop('teamID')
pass

In [ ]:
print(len(appearances))
appearances.head(2)

In [ ]:
mb_data = pd.merge(appearances, bbdata_sal)
print(len(mb_data))
mb_data.head(2)


In [ ]:
#ROSTER TIME / Rules of baseball ### Change Pitcher to DH 
ROSTER=['G_p', 'G_c', 'G_1b', 'G_2b','G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf']

In [ ]:
def mb_players(year, pos):
    return mb_data[(mb_data['yearID'] == year) & (mb_data[pos] > 5)][:100].sort(columns='OBP_sal_10k', ascending=False)

In [ ]:
def mb_roster(year):
    mbr_uf = []
    for pos in ROSTER:
        mbr_uf.append(mb_players(year, pos))
        
    ret = []
    no_overlap = []
    player_list = []
    for pos in mbr_uf:
        for n in range(len(pos)):
            if pos.iloc[n].playerID not in player_list:
                no_overlap.append(n)
        player_list.append(pos.iloc[no_overlap[0]].playerID)
        ret.append(pos.iloc[no_overlap[0]])
        no_overlap = []
                       
    return pd.DataFrame(ret)

In [1]:
#PLug in a year & Enjoy!!!!

mb_roster(2001)

NameError: name 'mb_roster' is not defined